# BIO 310 Homework 1 - Semi Global Sequence Alignment

In this homework, you will implement Semi Global Sequence Alignment. You are given a partial Global Alignment code. You are expected to modify some parts or fill in the blanks to implement it.

There are 2 main excercise that you will encounter in this Homework. For the detailed explanation, please refer to the Homework PDF.

* **MODIFY STARTS:** When you see MODIFY, it means this part is for Global Alignment. Therefore, you should convert it to Semi-Global Alignment

* **TODO STARTS:** You should implement this part or fill in the blank (...) parts for Semi-Global Alignments.

In [ ]:
# In Google Colab, Numpy and Pandas are installed already. In case you want to use them on your local computer, you can install them with pip

# !pip install numpy
# !pip install pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
 ###################### TODO START ######################

def initialize_penalties(match_score, mismatch_score, seq1_start_gap, seq2_start_gap,
                        seq1_end_gap, seq2_end_gap, internal_gap):
    match = match_score
    mismatch = mismatch_score

    gap_penalties = {
        'seq1_start': seq1_start_gap,
        'seq2_start': seq2_start_gap,
        'seq1_end': seq1_end_gap,
        'seq2_end': seq2_end_gap,
        'gap_internal': internal_gap
    }

    # The direction dictionary will be used for backtracking (Do not change)
    direction_dict = {
        'diagonal':0,
        'left': 1,
        'up':2
    }

    return match, mismatch, gap_penalties, direction_dict

 ###################### TODO END ######################

In [ ]:
def read_sequences(filename):
    """
    Read the file and return two sequences in list format

    Args:
        filename (string): name of the sequence file
    """
    with open(filename) as fn:
        seq1 = fn.readline().strip()
        seq2 = fn.readline().strip()
        match = int(fn.readline().split(':')[1].strip())
        mismatch = int(fn.readline().split(':')[1].strip())
        internal_gap = int(fn.readline().split(':')[1].strip())
        seq1_start_gap = int(fn.readline().split(':')[1].strip())
        seq2_start_gap = int(fn.readline().split(':')[1].strip())
        seq1_end_gap = int(fn.readline().split(':')[1].strip())
        seq2_end_gap = int(fn.readline().split(':')[1].strip())
    seq1 = list(seq1)
    seq2 = list(seq2)


    return seq1, seq2, match, mismatch, seq1_start_gap, seq2_start_gap, seq1_end_gap, seq2_end_gap, internal_gap

In [ ]:
 ###################### TODO START ######################

def isMatch(a,b, match_score, mismatch_score):
  if a == b:
    return match_score

  elif a != b:
    return mismatch_score
 ###################### TODO END ######################

In [ ]:
def score_matrix(seq1, seq2, match, mismatch, gap_penalties, direction_dict):
    """
    Creates score and direction matrices for alignment of two sequences

    Args:
        seq1 (list): DNA Sequence 1
        seq2 (list): DNA Sequence 2
        match_score (int): Match Score
        mismatch_score (int): Mismatch Score
        gap_penalties (dict): Contains different gap penalties

    Return:
        df_scores (pd.DataFrame): Score Matrix
        df_directions (pd.DataFrame): Direction Matrix
        max_loc (Tuple): Coordinates of the max position
    """

    len_seq1 = len(seq1)
    len_seq2 = len(seq2)

    # Zero-filled matrices for scores and directions
    scores = np.zeros((len_seq1 + 1, len_seq2 + 1))
    directions = np.zeros((len_seq1 + 1, len_seq2 + 1))

    ###################### MODIFY START ######################

    # Fill first row of scores and directions for seq 1 gap:
    for i in range(1, len(seq2) + 1):
        scores[0][i] = scores[0][i-1] + gap_penalties['seq1_start']
        directions[0][i] = direction_dict['left']

    # Fill first column of scores and directions for seq 2 gap:
    for j in range(1, len(seq1) + 1):
        scores[j][0] = scores[j-1][0] + gap_penalties['seq2_start']
        directions[j][0] = direction_dict['up']


    ###################### MODIFY END ######################


    # Fill the internal cells of scores and directions:
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):

            ###################### TODO START ######################

            # Calculate left score (Sequence 1 Gap), up score (Sequence 2 Gap), Diagonal Score (Match or Mismatch)
            left_score = scores[i][j-1] + gap_penalties['gap_internal']
            up_score = scores[i-1][j] + gap_penalties['gap_internal']

            diagonal_score = isMatch(seq1[i-1], seq2[j-1], match, mismatch)


            # Decide which way to go.
            move_score = max(left_score, up_score, diagonal_score)

            ###################### TODO END ######################

            if move_score == diagonal_score:
                scores[i][j] = diagonal_score
                directions[i][j] = direction_dict['diagonal']
            elif move_score == left_score:
                scores[i][j] = left_score
                directions[i][j] = direction_dict['left']
            else:
                scores[i][j] = up_score
                directions[i][j] = direction_dict['up']


    ###################### MODIFY START ######################

    # max_loc is the starting location of the backtracking.

    max_row = np.max(scores[ -1])
    max_column = np.max(scores[:, -1])
    max_score = max(max_row, max_column)


    for i in range(len(scores)):
      for j in range(len(scores[i])):
        if scores[i][j] == max_score:
          max_loc = [i,j]

    #print(max_loc)


    ###################### MODIFY END ######################


    # Our matrix size is (m+1, n+1). But our sequence lengths are m and n.
    # Add * at the beginnings of sequences
    if len(seq1) !=len(scores):
        seq1.insert(0,'*')
        seq2.insert(0,'*')
    # Convert them numpy array
    seq1_np = np.array(seq1)
    seq2_np = np.array(seq2)

    # To see better, convert them as pandas dataframe
    scores_matrix=np.asmatrix(scores)
    df_scores = pd.DataFrame(scores_matrix, columns= seq2_np, index = seq1_np)

    directions_matrix = np.asmatrix(directions)
    df_directions = pd.DataFrame(directions_matrix, columns= seq2_np, index = seq1_np)


    return df_scores, df_directions, max_loc


In [ ]:
def backtrack(scores, directions, max_loc):

    # Coordinates to track. Start from max_loc until first row or column
    x_track, y_track = max_loc[0], max_loc[1]

    # Length of Sequences
    seq1_length = len(scores.index)
    seq2_length = len(scores.columns)

    # Final informations to print the alignment
    sequence_1 = ''
    sequence_2 = ''
    alignment_info = ''

    ###################### TODO START ######################

    # Start from [x_track, y_track] and construct the alignment

    while x_track >= 0 and y_track >= 0:

        if directions.iloc[x_track, y_track] == 0: # Diagonal
            sequence_1 += scores.index[x_track]
            sequence_2 += scores.columns[y_track]

            if scores.index[x_track] == scores.columns[y_track]: # Match
                alignment_info += '|'
            else:
                alignment_info += '.'
            x_track = x_track - 1
            y_track = y_track - 1

        elif directions.iloc[x_track, y_track] == 1: # Left
            sequence_1 += '-'
            sequence_2 += scores.columns[y_track]
            alignment_info += ' '
            x_track = x_track
            y_track = y_track -1

        else: # Up
            sequence_1 += scores.index[x_track]
            sequence_2 += '-'
            alignment_info += ' '
            x_track = x_track -1
            y_track = y_track

    ###################### TODO END ######################


    ###################### TODO START ######################

    # Add the end gaps to the alignment

    while x_track >= 0:
      sequence_1 = scores.index[0:x_track+1][::-1] + sequence_1
      sequence_2 = '-' * (x_track +1) + sequence_2
      alignment_info = ' ' * (x_track+1) + alignment_info
      x_track -= 1 # to move next element

    while y_track >= 0:
      sequence_1 = '-' * (y_track+1) + sequence_1
      sequence_2 = scores.columns[0:y_track+1][::-1] + sequence_2
      alignment_info = ' '*(y_track+1) + alignment_info
      y_track -= 1 # to move to nex element

    ###################### TODO END ######################

    # Reverse the sequences
    sequence_1 = sequence_1[::-1]
    sequence_2 = sequence_2[::-1]
    alignment_info = alignment_info[::-1]

    return sequence_1, sequence_2, alignment_info


In [ ]:
###################### TODO START ######################

def semi_global_alignment(filename):

    # 1. Read File
    sequece_1, sequence_2, match_score, mismatch_score, seq1_start_gap, seq2_start_gap, seq1_end_gap, seq2_end_gap, internal_gap = read_sequences(filename)

    # 2. Initialize Penalties
    match_score, mismatch_score, gap_penalties, direction_dict = initialize_penalties(match_score, mismatch_score, seq1_start_gap, seq2_start_gap, seq1_end_gap, seq2_end_gap, internal_gap)

    # 3. Get score and direction matrix
    df_scores, df_directions, max_loc = score_matrix(sequece_1, sequence_2, match_score, mismatch_score, gap_penalties, direction_dict)

    # 4. Backtrack
    sequence_1, sequence_2, alignment_info = backtrack(df_scores, df_directions, max_loc)

    # 5. Print the results (Don't Change This)
    print(f'Match Score: {match_score}')
    print(f'Mismatch Score: {mismatch_score}')
    print(f'Internal Gap penalty: {internal_gap}')
    print(f'Gaps penalty at the start of string 1: {seq1_start_gap}')
    print(f'Gaps penalty at the start of string 2: {seq2_start_gap}')
    print(f'Gaps penalty at the end of string 1: {seq1_end_gap}')
    print(f'Gaps penalty at the end of string 2: {seq2_end_gap}')
    print()
    print(f'{sequence_1}\n{alignment_info}\n{sequence_2}')

###################### TODO END ######################

# Run the tests in the cells below

## Test 1

In [ ]:
semi_global_alignment('/content/drive/MyDrive/BIO310_2023_HW1_Test_Cases/test1.fasta')

Match Score: 15
Mismatch Score: -7
Internal Gap penalty: -7
Gaps penalty at the start of string 1: 0
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: 0
Gaps penalty at the end of string 2: 0

*ATTGCC--ATGA
|   | |  ||||
*---G-CATATGA


## Test 2

In [ ]:
semi_global_alignment('/content/drive/MyDrive/BIO310_2023_HW1_Test_Cases/test2.fasta')

Match Score: 10
Mismatch Score: -2
Internal Gap penalty: -2
Gaps penalty at the start of string 1: -8
Gaps penalty at the start of string 2: -8
Gaps penalty at the end of string 1: 0
Gaps penalty at the end of string 2: 0

*ATTGCC--ATGA
|   | |  ||||
*---G-CATATGA


## Test 3

In [ ]:
semi_global_alignment('/content/drive/MyDrive/BIO310_2023_HW1_Test_Cases/test3.fasta')

Match Score: 10
Mismatch Score: -3
Internal Gap penalty: -2
Gaps penalty at the start of string 1: 0
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: -10
Gaps penalty at the end of string 2: -10

*---ATCT-CTGATAAG--GACAAGG--CTG--CTGTGA--AA-GCC-CTATG-C
|    ||| |   |  |  ||||| |  |    |   |   || ||| | | | |
*CTG-TCTCC---T--GCCGACAA-GACC--AAC---G-TCAAGGCCGC-A-GCC


## Test 4

In [ ]:
semi_global_alignment('/content/drive/MyDrive/BIO310_2023_HW1_Test_Cases/test4.fasta')

Match Score: 10
Mismatch Score: -2
Internal Gap penalty: -7
Gaps penalty at the start of string 1: -7
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: -7
Gaps penalty at the end of string 2: 0

*ATCTCTGATAAGGACAAGGCTGCTGTG-AAA-GCC-CTATG-C
|.|.|||..|...||||| | ..|...| .|| ||| | | | |
*CTGTCTCCTGCCGACAA-G-ACCAACGTCAAGGCCGC-A-GCC


## Test 5

In [ ]:
semi_global_alignment('/content/drive/MyDrive/BIO310_2023_HW1_Test_Cases/test5.fasta')

Match Score: 5
Mismatch Score: -2
Internal Gap penalty: -2
Gaps penalty at the start of string 1: 0
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: 0
Gaps penalty at the end of string 2: 0

*CGATTTGATTGAG
|        ||  |
*--------TT--G


## Test 6

In [ ]:
semi_global_alignment('/content/drive/MyDrive/BIO310_2023_HW1_Test_Cases/test6.fasta')

Match Score: 1
Mismatch Score: -1
Internal Gap penalty: -2
Gaps penalty at the start of string 1: -1
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: 0
Gaps penalty at the end of string 2: 0

*CCAAGTCAAGTCGG
|  .....||...||
*--GTTCAAATCGGG


## Test 7

In [ ]:
semi_global_alignment('/content/drive/MyDrive/BIO310_2023_HW1_Test_Cases/test7.fasta')

Match Score: 1
Mismatch Score: -1
Internal Gap penalty: -1
Gaps penalty at the start of string 1: -1
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: 0
Gaps penalty at the end of string 2: 0

*BIOINFORMATICS
|   |||||||||||
*---INFORMATICS


## Test 8

In [ ]:
semi_global_alignment('/content/drive/MyDrive/BIO310_2023_HW1_Test_Cases/test8.fasta')

Match Score: 8
Mismatch Score: -2
Internal Gap penalty: -4
Gaps penalty at the start of string 1: -1
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: -1
Gaps penalty at the end of string 2: 0

*CATTCA-CAGGTCA
|    |  |  | ||
*----C-TC--G-CA


## Test 9

In [ ]:
semi_global_alignment('/content/drive/MyDrive/BIO310_2023_HW1_Test_Cases/test9.fasta')

Match Score: 10
Mismatch Score: -1
Internal Gap penalty: -2
Gaps penalty at the start of string 1: -2
Gaps penalty at the start of string 2: 0
Gaps penalty at the end of string 1: 0
Gaps penalty at the end of string 2: 0

*WEARE---HUMANS
||||||   ||||| 
*WEARENOTHUMAN-
